In [2]:
import pandas as pd

In [3]:
cpuc = pd.read_csv('data/cpuc_with_budget_and_resource.csv')

C:\Users\audre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,7,12,17,19,22,23,24,27,47,48,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
cpuc.columns

Index(['Unnamed: 0', 'ClaimID', 'PrgID', 'Sector_x', 'SiteCity', 'SiteZipCode',
       'SiteID', 'NAICSCode', 'BldgHVAC', 'BldgLoc', 'BldgType', 'BldgVint',
       'E3ClimateZone', 'E3GasSavProfile', 'E3GasSector',
       'E3MeaElecEndUseShape', 'E3TargetSector', 'ImplementationID',
       'InstallationDate', 'NAICSBldgType', 'NumUnits', 'OBF_Flag',
       'PrgElement', 'RateScheduleElec', 'RateScheduleGas', 'REN_Flag',
       'Residential_Flag', 'SchoolIdentifier', 'TotalFirstYearGrosskW',
       'TotalFirstYearGrosskWh', 'TotalFirstYearGrossTherm',
       'TotalGrossIncentive', 'TotalGrossMeasureCost',
       'TotalGrossMeasureCost_ER', 'TotalLifecycleGrosskW',
       'TotalLifecycleGrosskWh', 'TotalLifecycleGrossTherm', 'Upstream_Flag_x',
       'WaterOnly_Flag', 'PrgYear', 'Claim ID', 'Program ID',
       'First Year Gross kWh', 'First Year Gross Therm', 'Budget', 'PA',
       'ProgramName', 'Grouping', 'County', 'Selected', 'PrimarySector',
       'Sector_y', 'BP Sector', 'Program

In [5]:
cpucbudget = pd.read_csv('data/selected_programs_merged_with_budget.csv')

In [6]:
df = cpucbudget[['ClaimID', 'PrgID', 'SiteCity', 'TotalFirstYearGrosskWh', 'TotalFirstYearGrossTherm','TotalLifecycleGrosskWh', 'TotalLifecycleGrossTherm', 'Budget', 'Grouping', 'Resource_Flag','OBF_Flag']]

In [7]:
df.shape

(746373, 11)

making total claims column

In [8]:
l = [1]*746373

total_claims = pd.Series(l).rename('Total Claims')

df = df.join(total_claims)

Total Programs column 
# THIS NEEDS TO BE OF LOCAL GOVERNMENT PROGRAMS
updated version below

In [11]:
prg = df.groupby(['SiteCity', 'PrgID'], dropna = False).size().index.tolist() 
#need to count the number of times a city shows up 

a,b = [list(c) for c in zip(*prg)]

ccp = pd.Series(a) # made a series so can use value_counts

ccp = ccp.str.upper()

tp = ccp.value_counts(dropna = False) #total programs per city

print(prg[0:6])

tp.head()

[('ACAMPO', 'PGE21012'), ('ACAMPO', 'PGE211030'), ('ACTON', 'SCE-13-SW-002D'), ('ACTON', 'SCE-13-SW-002F'), ('ACTON', 'SCG3793'), ('ACTON', 'SCG3805')]


LOS ANGELES      70
ROSEMEAD         57
SAN FRANCISCO    47
SAN DIEGO        46
TORRANCE         24
dtype: int64

In [12]:
tp

LOS ANGELES      70
ROSEMEAD         57
SAN FRANCISCO    47
SAN DIEGO        46
TORRANCE         24
                 ..
CLEMENTS          1
LUDLOW            1
EL GRANADA        1
SHASTA LAKE       1
SUSANVILLE        1
Length: 1026, dtype: int64

making all site city upper case

In [13]:
df['SiteCity'] = df['SiteCity'].str.upper()

In [14]:
lg = ['Local Government (Counties)',
 'Local Government ()',
 'Local Government (Cities)',
 'Local Government ']

localgov = df[df['Grouping'].isin(lg)]

In [15]:
lg_city_agg = localgov.groupby('SiteCity').sum().join(tp.rename('Total Programs')).reset_index()

In [16]:
lg_city_agg[lg_city_agg['SiteCity'] == 'TWIN PEAKS']

,SiteCity,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Resource_Flag,OBF_Flag,Total Claims,Total Programs


local gov total programs

In [17]:
lgprg = localgov.groupby(['SiteCity', 'PrgID'], dropna = False).size().index.tolist() 
#need to count the number of times a city shows up 

e,f = [list(c) for c in zip(*lgprg)]

lgccp = pd.Series(e) # made a series so can use value_counts

lgccp = lgccp.str.upper()

lgtp = lgccp.value_counts(dropna = False) #total programs per city

lgtp.head()

ROSEMEAD         26
LOS ANGELES      26
SAN FRANCISCO    10
SAN DIEGO         8
FRESNO            5
dtype: int64

In [18]:
lg_city_agg = localgov.groupby('SiteCity').sum().join(lgtp.rename('Total Programs')).reset_index()

In [19]:
lg_city_agg.head()

,SiteCity,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Resource_Flag,OBF_Flag,Total Claims,Total Programs
0,ACAMPO,48664.726740,-148.890330,5.839767e+05,-1786.683960,17684.568509,6,0,6,1
1,ADELANTO,476437.532696,-90.111798,1.946481e+06,-450.558990,262068.388232,37,0,37,2
2,ALAMO,73049.392600,118.645240,6.850000e+05,-1279.097025,48785.542683,15,2,15,1
3,ALBANY,158184.149000,-818.167334,8.751087e+05,-4538.806680,63829.667146,40,0,40,2
4,ALHAMBRA,43261.898784,-64.344925,2.404667e+05,-357.759876,96930.895202,14,0,14,1


In [20]:
lg_city_agg.rename(columns={'OBF_Flag':'OBF Claims', 'Resource_Flag':'Resource Claims'}, inplace=True)

In [21]:

lg_city_agg['% OBF'] = (lg_city_agg['OBF Claims']/lg_city_agg['Total Claims'])*100

lg_city_agg['% Resource'] = (lg_city_agg['Resource Claims']/lg_city_agg['Total Claims'])*100



In [22]:
lg_city_agg.head()

,SiteCity,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Resource Claims,OBF Claims,Total Claims,Total Programs,% OBF,% Resource
0,ACAMPO,48664.726740,-148.890330,5.839767e+05,-1786.683960,17684.568509,6,0,6,1,0.000000,100.0
1,ADELANTO,476437.532696,-90.111798,1.946481e+06,-450.558990,262068.388232,37,0,37,2,0.000000,100.0
2,ALAMO,73049.392600,118.645240,6.850000e+05,-1279.097025,48785.542683,15,2,15,1,13.333333,100.0
3,ALBANY,158184.149000,-818.167334,8.751087e+05,-4538.806680,63829.667146,40,0,40,2,0.000000,100.0
4,ALHAMBRA,43261.898784,-64.344925,2.404667e+05,-357.759876,96930.895202,14,0,14,1,0.000000,100.0


In [23]:
lg_city_agg.to_csv('cpuc_city_agg.csv')

# the mystery of the discrepency in the number of claims

In [17]:
localgov['ClaimID'].value_counts()

SCE-2017-Q4-0362006    4
SCE-2017-Q4-0379686    4
PGE-2018-Q2-53857      4
SCE-2017-Q4-0379775    4
PGE-2018-Q2-53850      4
                      ..
PGE-2017-Q4-69739      1
PGE-2017-Q4-80007      1
PGE-2017-Q4-48056      1
PGE-2017-Q1-10443      1
PGE-2017-Q4-160100     1
Name: ClaimID, Length: 193253, dtype: int64

In [18]:
localgov.columns

Index(['ClaimID', 'PrgID', 'SiteCity', 'TotalFirstYearGrosskWh',
       'TotalFirstYearGrossTherm', 'TotalLifecycleGrosskWh',
       'TotalLifecycleGrossTherm', 'Budget', 'Grouping', 'Resource_Flag',
       'OBF_Flag', 'Total Claims'],
      dtype='object')

In [19]:
nodups = localgov.drop_duplicates()

In [20]:
nodups['ClaimID'].value_counts()

PGE-2017-Q4-51525      1
PGE-2018-Q4-93036      1
PGE-2018-Q3-52890      1
PGE-2018-Q1-48065      1
PGE-2017-Q4-40246      1
                      ..
SCE-2017-Q4-0445900    1
PGE-2018-Q1-38124      1
PGE-2017-Q4-14694      1
PGE-2018-Q4-75049      1
PGE-2017-Q4-160100     1
Name: ClaimID, Length: 193253, dtype: int64

# Aggregation Function

goal of this function is to have a reusable way to aggregate our data at a given level of speficity ie county and city.... esp because we will hopefully get data with a 'Total Expenditures' column, in which case we'll have to aggregate again...

the function will have a few parameters:
1. df, the dataframe needed for aggregation
1. by=str, the column to do the groupby aggregation, in this case will most likely be city or county
1. lg=True , filters out local government programs, if False, doesn't do any filtering
1. column_list, a list of relevant columns to be in the final aggregated dataframe
1. more things if I think of them

In [21]:
cpuc.head()

,Unnamed: 0,ClaimID,PrgID,Sector_x,SiteCity,SiteZipCode,SiteID,NAICSCode,BldgHVAC,BldgLoc,...,Custom_Flag,Upstream_Flag_y,Midstream_Flag,Downstream_Flag,DirectInstall,Audit_Flag,Financing,ParentProgram,Exclude_From_Budget,Exclude_From_CE
0,0,PGE-2017-Q4-85848,PGE21015,Com,PLEASANTON,94588,PGE-1118107,541512,cPVVG,CZ12,...,False,True,True,False,No,False,None,Commercial Energy Efficiency Programs,False,False
1,1,PGE-2017-Q4-94241,PGE211009,Res,WALNUT CREEK,94596,PGE-1133955,NaN,rWtd,CZ12,...,True,False,False,True,Partial,True,None,Government Partnership Programs,False,False
2,2,PGE-2017-Q4-88304,PGE211009,Res,UNION CITY,94587,PGE-1103730,NaN,rWtd,3A,...,True,False,False,True,Partial,True,None,Government Partnership Programs,False,False
3,3,PGE-2017-Q4-8624,PGE21012,Com,SAINT HELENA,94574,PGE-1082391,713910,cWtd,CZ02,...,False,True,True,True,No,True,None,Commercial Energy Efficiency Programs,False,False
4,4,PGE-2017-Q4-88820,PGE211009,Res,FREMONT,94539,PGE-1120910,NaN,rWtd,3B,...,True,False,False,True,Partial,True,None,Government Partnership Programs,False,False


goal is to go from a dataframe like the one above with all necesary columns, and spit out a relevant aggregation

### Making df Usable

In [ ]:
# making df usable

df[by] = df[by].str.upper()
df['ClaimID'].drop_duplicates()

if df['ClaimID'].value_counts()[0] != 1:
    print("Sorry your dataset contains duplicates I don't know how to deal with")
    return

### Local Government Filtering

In [ ]:
#filtering for local government

if lg==True: 
    
    lg = ['Local Government (Counties)',
     'Local Government ()',
     'Local Government (Cities)',
     'Local Government ']

    localgov = df[df['Grouping'].isin(lg)]
    

### Total Programs/Total Claims calculation

In [17]:
# creating a total programs column

prg = df.groupby([by, 'PrgID'], dropna = False).size().index.tolist() 
#need to count the number of times a city/county shows up 

e,f = [list(c) for c in zip(*prg)]

ccp = pd.Series(e) # made a series so can use value_counts

ccp = ccp.str.upper()

tp = ccp.value_counts(dropna = False) #total programs per city


ROSEMEAD         26
LOS ANGELES      26
SAN FRANCISCO    10
SAN DIEGO         8
FRESNO            5
dtype: int64

In [8]:
# creating a total claims column

l = [1]*len(df)
total_claims = pd.Series(l).rename('Total Claims')
df = df.join(total_claims)

### Actual Aggregation!

In [ ]:
#aggregate!
df.groupby(by).sum()[column_list]

---------
--------
------

In [25]:
def aggregate_me(df, by, lg, column_list):
    
    # making df usable

    df[by] = df[by].str.upper()
    df.drop_duplicates()

    if df['ClaimID'].value_counts()[0] != 1:
        print("Sorry your dataset contains duplicates I don't know how to deal with")
        return

    #filtering for local government

    if lg==True: 

        lg = ['Local Government (Counties)',
         'Local Government ()',
         'Local Government (Cities)',
         'Local Government ']

        localgov = df[df['Grouping'].isin(lg)]


    # creating a total programs column

    prg = df.groupby([by, 'PrgID'], dropna = False).size().index.tolist() 
    #need to count the number of times a city/county shows up 

    e,f = [list(c) for c in zip(*prg)]

    ccp = pd.Series(e) # made a series so can use value_counts

    ccp = ccp.str.upper()

    tp = ccp.value_counts(dropna = False) #total programs per city


    # creating a total claims column

    l = [1]*len(df)
    total_claims = pd.Series(l).rename('Total Claims')
    df = df.join(total_claims)

    #aggregate!
    df.groupby(by).sum()[column_list]

In [26]:
aggregate_me(cpuc,'SiteCity',True, ['Budget','Total Claims'])

Sorry your dataset contains duplicates I don't know how to deal with


# small digression into dealing with duplicates

In [27]:
cpuc['ClaimID'].drop_duplicates()

0                                   PGE-2017-Q4-85848
1                                   PGE-2017-Q4-94241
2                                   PGE-2017-Q4-88304
3                                    PGE-2017-Q4-8624
4                                   PGE-2017-Q4-88820
                             ...                     
746368               SDGE-2019-4061-11015387-12663812
746369    SDGE-2019-4061-2019Q1ProgramCostPlaceholder
746370    SDGE-2019-4061-2019Q2ProgramCostPlaceholder
746371    SDGE-2019-4061-2019Q3ProgramCostPlaceholder
746372    SDGE-2019-4061-2019Q4PROGRAMCOSTPLACEHOLDER
Name: ClaimID, Length: 599968, dtype: object

In [28]:
cpuc.drop_duplicates()

,Unnamed: 0,ClaimID,PrgID,Sector_x,SiteCity,SiteZipCode,SiteID,NAICSCode,BldgHVAC,BldgLoc,...,Custom_Flag,Upstream_Flag_y,Midstream_Flag,Downstream_Flag,DirectInstall,Audit_Flag,Financing,ParentProgram,Exclude_From_Budget,Exclude_From_CE
0,0,PGE-2017-Q4-85848,PGE21015,Com,PLEASANTON,94588,PGE-1118107,541512,cPVVG,CZ12,...,False,True,True,False,No,False,None,Commercial Energy Efficiency Programs,False,False
1,1,PGE-2017-Q4-94241,PGE211009,Res,WALNUT CREEK,94596,PGE-1133955,NaN,rWtd,CZ12,...,True,False,False,True,Partial,True,None,Government Partnership Programs,False,False
2,2,PGE-2017-Q4-88304,PGE211009,Res,UNION CITY,94587,PGE-1103730,NaN,rWtd,3A,...,True,False,False,True,Partial,True,None,Government Partnership Programs,False,False
3,3,PGE-2017-Q4-8624,PGE21012,Com,SAINT HELENA,94574,PGE-1082391,713910,cWtd,CZ02,...,False,True,True,True,No,True,None,Commercial Energy Efficiency Programs,False,False
4,4,PGE-2017-Q4-88820,PGE211009,Res,FREMONT,94539,PGE-1120910,NaN,rWtd,3B,...,True,False,False,True,Partial,True,None,Government Partnership Programs,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746368,746368,SDGE-2019-4061-11015387-12663812,SDGE4061,Commercial,SAN DIEGO,92123-1417,SDGE-2019-4061-11015387,522110,Any,CZ07,...,False,False,False,False,No,True,None,NaN,False,False
746369,746369,SDGE-2019-4061-2019Q1ProgramCostPlaceholder,SDGE4061,Commercial,SAN DIEGO,92123,SDGE-2019-4061-2019Q1,NaN,Any,CZ07,...,False,False,False,False,No,True,None,NaN,False,False
746370,746370,SDGE-2019-4061-2019Q2ProgramCostPlaceholder,SDGE4061,Commercial,SAN DIEGO,92123,SDGE-2019-4061-2019Q2,NaN,Any,CZ07,...,False,False,False,False,No,True,None,NaN,False,False
746371,746371,SDGE-2019-4061-2019Q3ProgramCostPlaceholder,SDGE4061,Commercial,SAN DIEGO,92123,SDGE-2019-4061-2019Q3,NaN,Any,CZ07,...,False,False,False,False,No,True,None,NaN,False,False


In [29]:
cpuc['ClaimID'].value_counts()

SCE-2017-Q4-0362024                4
SCE-2017-Q4-0362069                4
SCE-2017-Q4-0379570                4
SCE-2017-Q4-0379582                4
SCE-2017-Q4-0368786                4
                                  ..
SDGE-2018-3224-10896802-9048370    1
PGE-2017-Q2-48746                  1
PGE-2018-Q1-65414                  1
SCG-2019-3711-12184368-3335044     1
SDGE-2018-3224-10896803-9048466    1
Name: ClaimID, Length: 599968, dtype: int64

In [30]:
cpuc.drop(columns='Unnamed: 0', inplace=True)

In [32]:
cpuc.drop_duplicates()['ClaimID'].value_counts().head(10)

SCE-2017-Q4-0201694                 2
SCE-2018-Q4-0101513                 2
SCE-2017-Q4-0201431                 2
SCE-2018-Q4-0114147                 2
SCE-2018-Q2-0069119                 2
SCE-2019-Q2-0042829                 2
SCE-2017-Q2-0149031                 1
SCE-2017-Q4-0268537                 1
SDGE-2019-3224-11022181-12683213    1
PGE-2019-Q1-5802                    1
Name: ClaimID, dtype: int64

In [33]:
cpuc.columns

Index(['ClaimID', 'PrgID', 'Sector_x', 'SiteCity', 'SiteZipCode', 'SiteID',
       'NAICSCode', 'BldgHVAC', 'BldgLoc', 'BldgType', 'BldgVint',
       'E3ClimateZone', 'E3GasSavProfile', 'E3GasSector',
       'E3MeaElecEndUseShape', 'E3TargetSector', 'ImplementationID',
       'InstallationDate', 'NAICSBldgType', 'NumUnits', 'OBF_Flag',
       'PrgElement', 'RateScheduleElec', 'RateScheduleGas', 'REN_Flag',
       'Residential_Flag', 'SchoolIdentifier', 'TotalFirstYearGrosskW',
       'TotalFirstYearGrosskWh', 'TotalFirstYearGrossTherm',
       'TotalGrossIncentive', 'TotalGrossMeasureCost',
       'TotalGrossMeasureCost_ER', 'TotalLifecycleGrosskW',
       'TotalLifecycleGrosskWh', 'TotalLifecycleGrossTherm', 'Upstream_Flag_x',
       'WaterOnly_Flag', 'PrgYear', 'Claim ID', 'Program ID',
       'First Year Gross kWh', 'First Year Gross Therm', 'Budget', 'PA',
       'ProgramName', 'Grouping', 'County', 'Selected', 'PrimarySector',
       'Sector_y', 'BP Sector', 'ProgramImplementer', 

In [36]:
cpuc[cpuc['ClaimID']=='SCE-2017-Q4-0201694']

,ClaimID,PrgID,Sector_x,SiteCity,SiteZipCode,SiteID,NAICSCode,BldgHVAC,BldgLoc,BldgType,...,Custom_Flag,Upstream_Flag_y,Midstream_Flag,Downstream_Flag,DirectInstall,Audit_Flag,Financing,ParentProgram,Exclude_From_Budget,Exclude_From_CE
327677,SCE-2017-Q4-0201694,SCE-13-SW-002D,Com,CARSON,90746,SCE_00984706,722211,Any,CZ08,RFF,...,False,False,False,False,Yes,False,OBF,Statewide Commercial Energy Efficiency Program,False,False
327681,SCE-2017-Q4-0201694,SCE-13-SW-002D,Com,CARSON,90746,SCE_00984706,722211,Any,CZ08,RFF,...,False,False,False,False,Yes,False,OBF,Statewide Commercial Energy Efficiency Program,False,False


In [48]:
a = cpuc.iloc[327677, :].to_frame().transpose()
b = cpuc.iloc[327681, :].to_frame().transpose()

In [55]:
a[~a.isin(b)]

,ClaimID,PrgID,Sector_x,SiteCity,SiteZipCode,SiteID,NAICSCode,BldgHVAC,BldgLoc,BldgType,...,Custom_Flag,Upstream_Flag_y,Midstream_Flag,Downstream_Flag,DirectInstall,Audit_Flag,Financing,ParentProgram,Exclude_From_Budget,Exclude_From_CE
327677,SCE-2017-Q4-0201694,SCE-13-SW-002D,Com,CARSON,90746,SCE_00984706,722211,Any,CZ08,RFF,...,False,False,False,False,Yes,False,OBF,Statewide Commercial Energy Efficiency Program,False,False


In [50]:
same = a.merge(b, left_on='ClaimID', right_on='ClaimID',how='inner')

In [51]:
same.columns

Index(['ClaimID', 'PrgID_x', 'Sector_x_x', 'SiteCity_x', 'SiteZipCode_x',
       'SiteID_x', 'NAICSCode_x', 'BldgHVAC_x', 'BldgLoc_x', 'BldgType_x',
       ...
       'Custom_Flag_y', 'Upstream_Flag_y_y', 'Midstream_Flag_y',
       'Downstream_Flag_y', 'DirectInstall_y', 'Audit_Flag_y', 'Financing_y',
       'ParentProgram_y', 'Exclude_From_Budget_y', 'Exclude_From_CE_y'],
      dtype='object', length=143)

In [53]:
dif = same = a.merge(b, left_on='ClaimID', right_on='ClaimID',how='outer')

In [54]:
dif.columns

Index(['ClaimID', 'PrgID_x', 'Sector_x_x', 'SiteCity_x', 'SiteZipCode_x',
       'SiteID_x', 'NAICSCode_x', 'BldgHVAC_x', 'BldgLoc_x', 'BldgType_x',
       ...
       'Custom_Flag_y', 'Upstream_Flag_y_y', 'Midstream_Flag_y',
       'Downstream_Flag_y', 'DirectInstall_y', 'Audit_Flag_y', 'Financing_y',
       'ParentProgram_y', 'Exclude_From_Budget_y', 'Exclude_From_CE_y'],
      dtype='object', length=143)

## get rid of duplications function whee
going to have to manually look into value_counts of 'claimid' in order to determine the parameter n, the number of distinct claims that have duplicates.

In [60]:
#creating df of number of times a claim repeats

repeats = df['ClaimID'].value_counts().to_frame().reset_index()

In [63]:
repeats.head()

,index,ClaimID
0,SCE-2017-Q4-0362024,4
1,SCE-2017-Q4-0362069,4
2,SCE-2017-Q4-0379570,4
3,SCE-2017-Q4-0379582,4
4,SCE-2017-Q4-0368786,4


In [69]:
#getting list of duplicate claims

n=4
dupes=[]
for i in range(0,n+1):
    if repeats.loc[i, 'ClaimID']>1:
        dupes.append(repeats.loc[i, 'index'])
print(dupes)

['SCE-2017-Q4-0362024', 'SCE-2017-Q4-0362069', 'SCE-2017-Q4-0379570', 'SCE-2017-Q4-0379582', 'SCE-2017-Q4-0368786']


In [70]:
df[df['ClaimID'].isin(dupes)]

,ClaimID,PrgID,SiteCity,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Grouping,Resource_Flag,OBF_Flag,Total Claims
366661,SCE-2017-Q4-0362069,SCE-13-SW-002D,CARSON,402.325000,-1.85725,6034.875000,-27.858750,255.992514,NaN,True,False,1
366662,SCE-2017-Q4-0362069,SCE-13-SW-002D,CARSON,402.325000,-1.85725,6034.875000,-27.858750,255.992514,NaN,True,False,1
366663,SCE-2017-Q4-0362069,SCE-13-SW-002D,CARSON,402.325000,-1.85725,6034.875000,-27.858750,255.992514,NaN,True,False,1
366664,SCE-2017-Q4-0362069,SCE-13-SW-002D,CARSON,402.325000,-1.85725,6034.875000,-27.858750,255.992514,NaN,True,False,1
378070,SCE-2017-Q4-0362024,SCE-13-SW-002D,PARAMOUNT,159.600000,-0.18677,1424.430000,-1.669188,82.900487,NaN,True,False,1
378071,SCE-2017-Q4-0362024,SCE-13-SW-002D,PARAMOUNT,159.600000,-0.18677,1424.430000,-1.669188,82.900487,NaN,True,False,1
378072,SCE-2017-Q4-0362024,SCE-13-SW-002D,PARAMOUNT,159.600000,-0.18677,1424.430000,-1.669188,82.900487,NaN,True,False,1
378073,SCE-2017-Q4-0362024,SCE-13-SW-002D,PARAMOUNT,159.600000,-0.18677,1424.430000,-1.669188,82.900487,NaN,True,False,1
494486,SCE-2017-Q4-0379570,SCE-13-SW-002D,COVINA,20.710000,-0.03154,310.650000,-0.473100,43.521952,NaN,True,False,1
494487,SCE-2017-Q4-0379570,SCE-13-SW-002D,COVINA,20.710000,-0.03154,310.650000,-0.473100,43.521952,NaN,True,False,1


need to somehow store the first index value of each claim ID

could make list of indicies for each claimid and then take the lowest one, making sure there is still one instance of the claim id in the dataset

In [80]:
#getting one index value per duplicate claim

goodi=[]
for e in dupes:
    goodi.append(min(df[df['ClaimID'].isin([e])].index.to_list()))
print(goodi)

[378070, 366661, 494486, 500328, 500231]


In [92]:
# full list of duplicate claim indicies

alli=[]
for e in dupes:
    alli = alli + df[df['ClaimID'].isin([e])].index.to_list()
print(badi)


[378070, 378071, 378072, 378073, 366661, 366662, 366663, 366664, 494486, 494487, 494488, 494489, 500328, 500329, 500330, 500331, 500231, 500233, 500235, 500237]


In [93]:
type(alli)

list

In [95]:
# take goodi from alli
badi = [i for i in alli if i not in goodi]

In [96]:
len(badi)

15

In [97]:
len(goodi)

5

In [98]:
len(alli)

20

In [99]:
#drop duplicate indicies
df.drop(index=badi)

,ClaimID,PrgID,SiteCity,TotalFirstYearGrosskWh,TotalFirstYearGrossTherm,TotalLifecycleGrosskWh,TotalLifecycleGrossTherm,Budget,Grouping,Resource_Flag,OBF_Flag,Total Claims
0,PGE-2017-Q4-85848,PGE21015,PLEASANTON,17006.650170,1790.515,56121.945550,5908.6995,5541.932761,NaN,True,False,1
1,PGE-2017-Q4-94241,PGE211009,WALNUT CREEK,35.040000,-0.832,560.640000,-13.3120,106.846221,Local Government (Counties),True,False,1
2,PGE-2017-Q4-88304,PGE211009,UNION CITY,13.140000,-0.312,210.240000,-4.9920,39.779768,Local Government (Counties),True,False,1
3,PGE-2017-Q4-8624,PGE21012,SAINT HELENA,0.000000,643.678,0.000000,12873.5600,1354.671628,NaN,True,False,1
4,PGE-2017-Q4-88820,PGE211009,FREMONT,35.040000,-0.832,560.640000,-13.3120,106.118823,Local Government (Counties),True,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...
746368,SDGE-2019-4061-11015387-12663812,SDGE4061,SAN DIEGO,2533.022933,0.000,5066.045866,0.0000,748.712453,NaN,True,False,1
746369,SDGE-2019-4061-2019Q1ProgramCostPlaceholder,SDGE4061,SAN DIEGO,0.000000,0.000,0.000000,0.0000,0.000000,NaN,True,False,1
746370,SDGE-2019-4061-2019Q2ProgramCostPlaceholder,SDGE4061,SAN DIEGO,0.000000,0.000,0.000000,0.0000,0.000000,NaN,True,False,1
746371,SDGE-2019-4061-2019Q3ProgramCostPlaceholder,SDGE4061,SAN DIEGO,0.000000,0.000,0.000000,0.0000,0.000000,NaN,True,False,1


In [101]:
df.drop(labels=badi, axis=0)['ClaimID'].value_counts()

SCE-2017-Q4-0365983                4
SCE-2017-Q4-0362042                4
SCE-2017-Q4-0365958                4
SCE-2017-Q4-0379696                4
SCE-2017-Q4-0361149                4
                                  ..
PGE-2017-Q2-48746                  1
PGE-2018-Q1-65414                  1
SCG-2019-3711-12184368-3335044     1
PGE-2018-Q4-87850                  1
SDGE-2018-3224-10896803-9048466    1
Name: ClaimID, Length: 599968, dtype: int64

In [ ]:
def no_dupes(df):
    
    #creating df of number of times a claim repeats

    repeats = df['ClaimID'].value_counts().to_frame().reset_index()

    #getting list of duplicate claims
    
    all_dupes = repeats[repeats['ClaimID']>1]
    n = len(all_dupes)
    dupes=[]
    for i in range(0,n):
        if repeats.loc[i, 'ClaimID']>1:
            dupes.append(repeats.loc[i, 'index'])
    print(dupes)

    #getting one index value per duplicate claim

    goodi=[]
    for e in dupes:
        goodi.append(min(df[df['ClaimID'].isin([e])].index.to_list()))
    print(goodi)

    # full list of duplicate claim indicies

    alli=[]
    for e in dupes:
        alli = alli + df[df['ClaimID'].isin([e])].index.to_list()
    print(badi)


    # take goodi from alli
    badi = [i for i in alli if i not in goodi]

    #drop duplicate indicies
    df.drop(index=badi, inplace=True)

In [102]:
copy = cpuc.copy()

In [111]:
copy['ClaimID'].value_counts().head(1000)

SCE-2017-Q4-0362024    4
SCE-2017-Q4-0362069    4
SCE-2017-Q4-0379570    4
SCE-2017-Q4-0379582    4
SCE-2017-Q4-0368786    4
                      ..
PGE-2017-Q1-42461      3
SCE-2018-Q3-0000425    3
PGE-2018-Q2-31951      3
SCE-2017-Q1-0068701    3
SCE-2018-Q2-0071312    3
Name: ClaimID, Length: 1000, dtype: int64

In [114]:
all_dupes = repeats[repeats['ClaimID']>1]
n = len(all_dupes)

138010

In [ ]:
no_dupes(copy)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

